# Customer Segmentation using PCA

This notebook demonstrates how to perform customer segmentation using **Principal Component Analysis (PCA)** for dimensionality reduction followed by **K-Means clustering**. It walks through synthetic data generation, preprocessing, PCA, clustering, visualization, and interpretation for teaching or practical use.

## Objectives

- Generate a synthetic customer dataset with realistic behavioral features (RFM + behavior).
- Scale features and apply PCA to reduce dimensionality.
- Use K-Means on the PCA-reduced data to obtain customer segments.
- Visualize clusters in 2D (first two principal components), inspect explained variance, and profile clusters in original feature space.
- Provide guidance on selecting number of components and clusters.


In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.datasets import make_blobs

plt.rcParams.update({'figure.max_open_warning': 0})

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [23]:
# Generate synthetic customer dataset
# Features: recency (days), frequency (#orders), monetary (total spend),
# avg_order_value, tenure (months), visits_per_month, promo_response_rate
n_customers = 1500

# Define archetypal cluster centers (realistic ranges)
centers = [
    [20, 50, 5000, 100, 60, 10, 0.6],     # high-value loyal
    [120, 10, 800, 80, 24, 2, 0.2],       # occasional mid-value
    [10, 2, 150, 75, 3, 1, 0.1],          # new low-value
    [200, 5, 300, 60, 48, 1.5, 0.8]       # churn-risk/discount seekers
]
cluster_std = [10, 20, 3, 40]

# Generate blobs in a latent 2D space and map to realistic features
X_blob, y_blob = make_blobs(n_samples=n_customers, centers=len(centers), cluster_std=cluster_std, random_state=RANDOM_SEED)

def map_blob_to_features(z, center_idx):
    # Map 2D blob coordinate to the 7 features by perturbing the archetypal center
    cx = centers[center_idx]
    noise = np.random.normal(scale=1.0, size=len(cx))
    mapped = np.array(cx) + noise + 0.05 * np.repeat(z, 4)[:len(cx)]
    return mapped

rows = []
for xi, lbl in zip(X_blob, y_blob):
    rows.append(map_blob_to_features(xi, lbl))

cols = ['recency_days','frequency_orders','monetary_total','avg_order_value','tenure_months','visits_per_month','promo_response_rate']

df = pd.DataFrame(rows, columns=cols)
for c in cols:
    df[c] = df[c].clip(lower=0.01)

df['customer_id'] = ['C{:05d}'.format(i) for i in range(len(df))]
df = df.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

print('Dataset shape:', df.shape)
df.head().round(2)


Dataset shape: (1500, 8)


,recency_days,frequency_orders,monetary_total,avg_order_value,tenure_months,visits_per_month,promo_response_rate,customer_id
0,201.41,5.86,301.37,61.41,45.32,0.01,0.01,C01116
1,117.43,9.80,797.96,78.71,25.87,1.00,0.01,C01368
2,8.86,3.55,150.74,74.57,3.78,0.01,0.01,C00422
3,118.08,10.98,800.42,80.72,30.17,2.16,4.17,C00413
4,200.54,6.76,300.74,59.83,49.63,2.22,1.28,C00451


In [24]:
# Preprocessing: scale features and run PCA
features = cols
scaler = StandardScaler()
X = scaler.fit_transform(df[features])

# PCA: fit full PCA and inspect explained variance
pca = PCA()
X_pca = pca.fit_transform(X)
explained = pca.explained_variance_ratio_
cum_explained = np.cumsum(explained)

print('Explained variance ratio (first 6):', np.round(explained[:6], 3))
print('Cumulative explained variance (first 6):', np.round(cum_explained[:6], 3))

# Choose number of components (simple rule: keep components that explain 90% variance)
k_components = int(np.searchsorted(cum_explained, 0.90) + 1)
print('Selected k_components (>=90% variance):', k_components)

# For visualization keep 2 components
X_pca_2 = X_pca[:, :2]

# attach to dataframe for plotting
df['PC1'] = X_pca_2[:,0]
df['PC2'] = X_pca_2[:,1]

# Save scaled data for later
scaled_df = pd.DataFrame(X, columns=features)
scaled_df['customer_id'] = df['customer_id']


Explained variance ratio (first 6): [0.628 0.25  0.097 0.016 0.008 0.   ]
Cumulative explained variance (first 6): [0.628 0.878 0.975 0.991 0.999 1.   ]
Selected k_components (>=90% variance): 3


In [25]:
# Cluster in PCA space using KMeans and evaluate (silhouette score)
sil_scores = []
K_range = range(2,7)
for K in K_range:
    kmeans = KMeans(n_clusters=K, random_state=RANDOM_SEED, n_init=10)
    labels = kmeans.fit_predict(X_pca_2)
    score = silhouette_score(X_pca_2, labels)
    sil_scores.append(score)

best_K = K_range[np.argmax(sil_scores)]
print('Silhouette scores:', dict(zip(K_range, np.round(sil_scores,3))))
print('Selected K (best silhouette):', best_K)

# Fit final KMeans
kmeans_final = KMeans(n_clusters=best_K, random_state=RANDOM_SEED, n_init=10)
labels_final = kmeans_final.fit_predict(X_pca_2)
df['cluster'] = labels_final

# cluster centers in PCA space
centers_pca = kmeans_final.cluster_centers_


Silhouette scores: {2: np.float64(0.703), 3: np.float64(0.676), 4: np.float64(0.709), 5: np.float64(0.722), 6: np.float64(0.704)}
Selected K (best silhouette): 5


In [26]:
# Visualize clusters in PCA 2D space
fig, ax = plt.subplots(figsize=(6,5))
scatter = ax.scatter(df['PC1'], df['PC2'], c=df['cluster'], cmap='tab10', s=30, alpha=0.7)
ax.scatter(centers_pca[:,0], centers_pca[:,1], marker='X', s=200, c='black', label='centroids')
ax.set_title('Customer segments in PCA-reduced 2D space')
ax.set_xlabel('PC1'); ax.set_ylabel('PC2')
ax.legend()
plt.tight_layout()
cluster_scatter_path = 'pca_clusters_scatter.png'
fig.savefig(cluster_scatter_path)
plt.close(fig)
cluster_scatter_path


'pca_clusters_scatter.png'

In [27]:
# Scree plot: explained variance ratio
fig, ax = plt.subplots(figsize=(5,3))
ax.plot(range(1, len(explained)+1), explained, marker='o')
ax.set_xlabel('Component index')
ax.set_ylabel('Explained variance ratio')
ax.set_title('Scree plot (explained variance)')
plt.tight_layout()
scree_path = 'pca_scree.png'
fig.savefig(scree_path)
plt.close(fig)
scree_path


'pca_scree.png'

In [28]:
# Profile clusters in original feature space (means)
cluster_profiles = df.groupby('cluster')[features].mean()
cluster_sizes = df['cluster'].value_counts().sort_index()

print('Cluster sizes:\n', cluster_sizes)
cluster_profiles_round = cluster_profiles.round(2)
cluster_profiles_round

# Bar chart of cluster centers (mean feature values)
fig, ax = plt.subplots(figsize=(8,4))
cluster_profiles_round.T.plot(kind='bar', ax=ax)
ax.set_title('Cluster profiles (mean feature values)')
ax.set_xlabel('Feature')
ax.set_ylabel('Mean (original scale)')
plt.tight_layout()
profile_path = 'cluster_profiles.png'
fig.savefig(profile_path)
plt.close(fig)
profile_path


Cluster sizes:
 cluster
0    369
1    375
2    259
3    374
4    123
Name: count, dtype: int64


'cluster_profiles.png'

## Interpretation & next steps

- Examine the cluster profiles and sizes to label segments (e.g., High-value loyal, Occasional shoppers, New customers, Discount-seekers).
- Validate segments against business KPIs (LTV, churn, response rate).
- Use the PCA+KMeans pipeline to score new customers (transform & predict) and target marketing.
- Consider alternative clustering (Gaussian Mixture Models) or including temporal features for lifecycle analysis.

### Exercises for students
1. Try clustering on original scaled features (without PCA). How do clusters change?
2. Use different scalers (RobustScaler) if features have heavy tails.
3. Try using more components (k_components chosen earlier) and re-run clustering.
